In [ ]:
import sys
import os

# Assuming your project directory is one level up from the Jupyter notebook
sys.path.insert(0, os.path.abspath('..'))

import lightning as L
import matplotlib
import numpy as np
import pandas as pd
import multiprocessing
from src.pipelines.trainers.trainerWrapper import TrainerWrapper
from src.util.conditional_early_stopping import ConditionalEarlyStopping
from src.util.flex_error import get_mafe, get_prob_mafe
from src.util.plot import plot_results
from src.util.power_splitter import PowerSplitter
from src.util.error import NRMSE

from src.pipelines.cleaners.temp_cleaner import TempCleaner
from src.pipelines.models.gru import GRU
from src.pipelines.normalizers.min_max_normalizer import MinMaxNormalizer
from src.pipelines.sequencers.time_sequencer import TimeSequencer
from src.pipelines.splitters.std_splitter import StdSplitter
from src.pipelines.tuners.std_tuner_wrapper import StdTunerWrapper
from src.pipelines.optimizers.optimizer import OptimizerWrapper

from src.pipelines.deterministic_pipeline import DeterministicPipeline
from src.pipelines.monte_carlo_pipeline import MonteCarloPipeline
from src.pipelines.ensemble_pipeline import EnsemblePipeline
from src.pipelines.probabilistic_pipeline import ProbabilisticPipeline

import torch.optim as optim

matplotlib.use("Agg")

MODEL_PATH = 'model.pth'
NUM_WORKERS = multiprocessing.cpu_count()
TARGET_COLUMN = 2
TIMESTAMP = "Timestamp"
POWER     = "PowerConsumption"

# Hyper parameters
# Model
input_size = 4
time_horizon = 4
hidden_size = 32
num_epochs = 1000
seq_len = 96
num_layers = 2
 
# MC ONLY
inference_samples = 50

# Training
dropout = 0.50
gradient_clipping = 0
early_stopping_threshold = 0.15

num_ensembles = 5

# Flexibility
flex_confidence = 0.90
temp_boundary = 0.1
error = 0

# Controlled by tuner
batch_size = 128
learning_rate = 0.005

# Data Split
train_days = 16
val_days = 2
test_days = 2

# ON / OFF Power Limits
off_limit_w = 100
on_limit_w = 1500

consecutive_points = 3

nist_path = "../src/data_preprocess/dataset/NIST_cleaned.csv"

clean_in_low = 10
clean_in_high = 30
clean_out_low = -50
clean_out_high = 50
clean_pow_low = 0
clean_delta_temp = 15

assert time_horizon > 0, "Time horizon must be a positive integer"
    
df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = StdSplitter(train_days, val_days, test_days)
    
model = GRU(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[ConditionalEarlyStopping(threshold=early_stopping_threshold)], 
                         gradient_clip_val=gradient_clipping)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(TimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_trainer(trainer) \
    .set_tuner_class(StdTunerWrapper) \
    .build()

model.fit()
model.test()

plot_results(model.get_predictions(), model.get_actuals(), model.get_timestamps())

model.eval()

ps = PowerSplitter(splitter.get_test(cleaner.clean(df)), TIMESTAMP, POWER)

on_df = ps.get_mt_power(on_limit_w, consecutive_points)
off_df = ps.get_lt_power(off_limit_w, consecutive_points)

on_data = np.array(on_df)
off_data = np.array(off_df)

print("Calculating mafe...")
if (isinstance(model, ProbabilisticPipeline)):
    print("PROB MAFE ON:", get_prob_mafe(on_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN, flex_confidence))
    print("PROB MAFE OFF:", get_prob_mafe(off_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN, flex_confidence))
else:
    print("MAFE ON:", get_mafe(on_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN))
    print("MAFE OFF:", get_mafe(off_data, model, seq_len, error, temp_boundary, time_horizon, TARGET_COLUMN))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Finding best initial lr: 100%|██████████| 100/100 [00:01<00:00, 64.34it/s]
Learning rate set to 4.786300923226383e-07
Restoring states from the checkpoint path at /home/vind/P5/experimentation/experiments/.lr_find_840ec486-c96a-4ed3-89e6-d487f361fa7e.ckpt
Restored all states from the checkpoint at /home/vind/P5/experimentation/experiments/.lr_find_840ec486-c96a-4ed3-89e6-d487f361fa7e.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 2 succeeded, trying batch size 4
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 4 succeeded, trying batch size 8
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 8 succeeded, trying batch size 16
`Trainer.fit` stopped: `max_steps=3` reached.
Batch size 16 succeeded, trying batch size 32
`Trainer.fit` stopped: `max_steps=3` reached.

Epoch 27: 100%|██████████| 71/71 [00:01<00:00, 50.26it/s, v_num=15, train_loss_step=0.113, val_loss=0.090, train_loss_epoch=0.126]  


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 9/9 [00:00<00:00, 93.02it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        0.08603551238775253
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Calculating mafe...
data: [[Timestamp('2022-07-02 01:00:00+0000', tz='UTC') 1741.3571428551222
  19.77 12.31]
 [Timestamp('2022-07-02 01:15:00+0000', tz='UTC') 3188.357142855238 19.7
  12.22]
 [Timestamp('2022-07-02 01:30:00+0000', tz='UTC') 2405.142857143801 19.7
  12.11]
 [Timestamp('2022-07-03 01:15:00+0000', tz='UTC') 3311.285714284168 19.6
  12.13]
 [Timestamp('2022-07-03 01:30:00+0000', tz='UTC') 2420.2142857138824 19.5
  11.75]
 [Timestamp('2022-07-03 01:45:00+0000